In [ ]:
!pip install ydata-profiling
!pip install --upgrade Pillow

In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import os as os

Read dataset

In [ ]:
df = pd.read_csv("CARES_data_cleaned.csv")
df

,Indexno,Age,Gender,RCRI score,Anemia category,PreopEGFRMDRD,GradeofKidneydisease,DaysbetweenDeathandoperation,@30daymortality,Preoptransfusionwithin30days,...,Anemiacategorybinned_Moderate/Severe,Anemiacategorybinned_NA,RDW15.7_<= 15.7,RDW15.7_>15.7,RDW15.7_NA,ASAcategorybinned_I,ASAcategorybinned_II,ASAcategorybinned_III,ASAcategorybinned_IV-VI,ASAcategorybinned_NA
0,2,48.0,Female,NaN,NaN,NaN,NaN,NaN,False,0.0,...,False,True,False,False,True,True,False,False,False,False
1,5,36.0,Female,NaN,NaN,NaN,NaN,NaN,False,0.0,...,False,True,True,False,False,True,False,False,False,False
2,6,64.0,Female,NaN,Mild,152.538570,G1,NaN,False,0.0,...,False,False,True,False,False,True,False,False,False,False
3,9,73.0,Male,NaN,Moderate,117.231496,G1,NaN,False,0.0,...,True,False,True,False,False,True,False,False,False,False
4,10,73.0,Male,0.0,Mild,98.651255,G1,59.0,False,0.0,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90780,121192,66.0,Female,2.0,Mild,48.339582,G3a,NaN,False,0.0,...,False,False,True,False,False,False,True,False,False,False
90781,121194,50.0,Male,1.0,Moderate,126.592489,G1,NaN,False,0.0,...,True,False,True,False,False,False,False,False,False,True
90782,121197,58.0,Female,NaN,NaN,86.306771,G2,NaN,False,0.0,...,False,True,True,False,False,False,False,False,False,True
90783,121198,63.0,Female,0.0,NaN,NaN,NaN,NaN,False,0.0,...,False,True,False,False,True,False,True,False,False,False


In [ ]:
df.describe()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop
count,90785.000000,90785.000000,63361.000000,79955.000000,5595.000000,90785.000000,90785.000000,90785.000000,90785.000000
mean,60334.001057,52.253225,0.322296,96.426155,476.782127,0.042672,0.056276,0.018946,0.075222
std,34930.467992,17.087307,0.634789,33.954241,421.866740,0.405840,0.230455,0.260728,0.395623
min,2.000000,18.000000,0.000000,2.541026,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29973.000000,39.000000,0.000000,79.078715,121.000000,0.000000,0.000000,0.000000,0.000000
50%,60572.000000,54.000000,0.000000,96.398561,355.000000,0.000000,0.000000,0.000000,0.000000
75%,90788.000000,65.000000,1.000000,114.310169,746.000000,0.000000,0.000000,0.000000,0.000000
max,121200.000000,103.000000,6.000000,671.298147,1783.000000,21.000000,1.000000,23.000000,24.000000


In [ ]:
# Check number of missing values in each column
missing_values = df.isna().sum()
print(missing_values)

Indexno                        0
Age                            0
Gender                         0
RCRI score                 27424
Anemia category            66916
                           ...  
ASAcategorybinned_I            0
ASAcategorybinned_II           0
ASAcategorybinned_III          0
ASAcategorybinned_IV-VI        0
ASAcategorybinned_NA           0
Length: 94, dtype: int64


In [ ]:
# Convert the result to a DataFrame for better visualization
missing_values_df = missing_values.reset_index()
missing_values_df.columns = ['Column', 'Missing Values']

display(missing_values_df)

,Column,Missing Values
0,Indexno,0
1,Age,0
2,Gender,0
3,RCRI score,27424
4,Anemia category,66916
...,...,...
89,ASAcategorybinned_I,0
90,ASAcategorybinned_II,0
91,ASAcategorybinned_III,0
92,ASAcategorybinned_IV-VI,0


In [ ]:
# Convert the result to a DataFrame for better visualization
missing_values_df = missing_values.reset_index()
missing_values_df.columns = ['Column', 'Missing Values']

# Export to Excel
missing_values_df.to_excel('missing_values.xlsx', index=False)

Remove columns that have been one-hot encoded, to prevent multicollinearity issues

In [ ]:
df.drop(['Gender', 'Anemia category', 'GradeofKidneydisease', 'AnaestypeCategory', 'PriorityCategory',
        'TransfusionIntraandpostopCategory', 'AGEcategory', 'AGEcategoryOriginal', 'SurgRiskCategory',
       'RaceCategory', 'GradeofKidneyCategory', 'Anemiacategorybinned', 'RDW15.7', 'ASAcategorybinned'],
       axis=1, inplace = True)

In [ ]:
df.head()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,@30daymortality,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop,...,Anemiacategorybinned_Moderate/Severe,Anemiacategorybinned_NA,RDW15.7_<= 15.7,RDW15.7_>15.7,RDW15.7_NA,ASAcategorybinned_I,ASAcategorybinned_II,ASAcategorybinned_III,ASAcategorybinned_IV-VI,ASAcategorybinned_NA
0,2,48.0,NaN,NaN,NaN,False,0.0,0.0,0.0,0.0,...,False,True,False,False,True,True,False,False,False,False
1,5,36.0,NaN,NaN,NaN,False,0.0,0.0,0.0,0.0,...,False,True,True,False,False,True,False,False,False,False
2,6,64.0,NaN,152.538570,NaN,False,0.0,0.0,0.0,0.0,...,False,False,True,False,False,True,False,False,False,False
3,9,73.0,NaN,117.231496,NaN,False,0.0,1.0,0.0,1.0,...,True,False,True,False,False,True,False,False,False,False
4,10,73.0,0.0,98.651255,59.0,False,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,True,False,False,False


In [ ]:
# Read original dataset first
df = pd.read_csv("CARES_data_cleaned.csv")
df

,Indexno,Age,Gender,RCRI score,Anemia category,PreopEGFRMDRD,GradeofKidneydisease,DaysbetweenDeathandoperation,@30daymortality,Preoptransfusionwithin30days,...,Anemiacategorybinned_Moderate/Severe,Anemiacategorybinned_NA,RDW15.7_<= 15.7,RDW15.7_>15.7,RDW15.7_NA,ASAcategorybinned_I,ASAcategorybinned_II,ASAcategorybinned_III,ASAcategorybinned_IV-VI,ASAcategorybinned_NA
0,2,48.0,Female,NaN,NaN,NaN,NaN,NaN,False,0.0,...,False,True,False,False,True,True,False,False,False,False
1,5,36.0,Female,NaN,NaN,NaN,NaN,NaN,False,0.0,...,False,True,True,False,False,True,False,False,False,False
2,6,64.0,Female,NaN,Mild,152.538570,G1,NaN,False,0.0,...,False,False,True,False,False,True,False,False,False,False
3,9,73.0,Male,NaN,Moderate,117.231496,G1,NaN,False,0.0,...,True,False,True,False,False,True,False,False,False,False
4,10,73.0,Male,0.0,Mild,98.651255,G1,59.0,False,0.0,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90780,121192,66.0,Female,2.0,Mild,48.339582,G3a,NaN,False,0.0,...,False,False,True,False,False,False,True,False,False,False
90781,121194,50.0,Male,1.0,Moderate,126.592489,G1,NaN,False,0.0,...,True,False,True,False,False,False,False,False,False,True
90782,121197,58.0,Female,NaN,NaN,86.306771,G2,NaN,False,0.0,...,False,True,True,False,False,False,False,False,False,True
90783,121198,63.0,Female,0.0,NaN,NaN,NaN,NaN,False,0.0,...,False,True,False,False,True,False,True,False,False,False


The mean of the **RCRI score** decreased (from 0.322 to 0.225) after replacing missing values in the **RCRI score** column with the median.

In [ ]:
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())


27424
10830


In [ ]:
"""
mean_PreopEGFRMDRD = df['PreopEGFRMDRD'].mean()
mean_PreopEGFRMDRD
df['PreopEGFRMDRD'].fillna(mean_PreopEGFRMDRD, inplace = True)
"""
# Interpolation imputation for RCRI score and PreopEGFRMDRD
df['RCRI score'].interpolate(method='linear', inplace=True)
df['PreopEGFRMDRD'].interpolate(method='linear', inplace=True)

# Forward fill to handle missing values at the start
df['RCRI score'].fillna(method='ffill', inplace=True)
df['PreopEGFRMDRD'].fillna(method='ffill', inplace=True)

# Backward fill to handle missing values at the end
df['RCRI score'].fillna(method='bfill', inplace=True)
df['PreopEGFRMDRD'].fillna(method='bfill', inplace=True)


<ipython-input-45-ba73b0d8459b>:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['RCRI score'].fillna(method='ffill', inplace=True)
<ipython-input-45-ba73b0d8459b>:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PreopEGFRMDRD'].fillna(method='ffill', inplace=True)
<ipython-input-45-ba73b0d8459b>:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['RCRI score'].fillna(method='bfill', inplace=True)
<ipython-input-45-ba73b0d8459b>:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PreopEGFRMDRD'].fillna(method='bfill', inplace=True)


In [ ]:
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())

0
0


In [ ]:
df.describe()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop
count,90785.000000,90785.000000,90785.000000,90785.000000,5595.000000,90785.000000,90785.000000,90785.000000,90785.000000
mean,60334.001057,52.253225,0.315999,97.787768,476.782127,0.042672,0.056276,0.018946,0.075222
std,34930.467992,17.087307,0.587262,33.144519,421.866740,0.405840,0.230455,0.260728,0.395623
min,2.000000,18.000000,0.000000,2.541026,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29973.000000,39.000000,0.000000,80.655950,121.000000,0.000000,0.000000,0.000000,0.000000
50%,60572.000000,54.000000,0.000000,98.016577,355.000000,0.000000,0.000000,0.000000,0.000000
75%,90788.000000,65.000000,0.500000,115.528070,746.000000,0.000000,0.000000,0.000000,0.000000
max,121200.000000,103.000000,6.000000,671.298147,1783.000000,21.000000,1.000000,23.000000,24.000000


# Interpolation imputation with Backward fill to handle missing values at the end and Forward fill imputations to handle missing values at the start

The mean of the **RCRI score** almost remained the same from 0.322 to 0.316

The mean of the **PreopEGFRMDRD** slightly increased from 96.426 to 97.788

In [ ]:
# Read original dataset first
df = pd.read_csv("CARES_data_cleaned.csv")
df

#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())


27424
10830


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
# Using MICE for imputation
mice_imputer = IterativeImputer()
df_imputed = df.copy()
imputed_data = mice_imputer.fit_transform(df[['RCRI score', 'PreopEGFRMDRD']])

# Replace NaN values with computed values
df['RCRI score'] = imputed_data[:, 0]
df['PreopEGFRMDRD'] = imputed_data[:, 1]

In [ ]:
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())

0
0


In [ ]:
df.describe()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop
count,90785.000000,90785.000000,90785.000000,90785.000000,5595.000000,90785.000000,90785.000000,90785.000000,90785.000000
mean,60334.001057,52.253225,0.321472,96.703195,476.782127,0.042672,0.056276,0.018946,0.075222
std,34930.467992,17.087307,0.541184,31.970736,421.866740,0.405840,0.230455,0.260728,0.395623
min,2.000000,18.000000,-1.994679,2.541026,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29973.000000,39.000000,0.000000,81.704788,121.000000,0.000000,0.000000,0.000000,0.000000
50%,60572.000000,54.000000,0.000000,96.703195,355.000000,0.000000,0.000000,0.000000,0.000000
75%,90788.000000,65.000000,0.410212,111.439680,746.000000,0.000000,0.000000,0.000000,0.000000
max,121200.000000,103.000000,6.000000,671.298147,1783.000000,21.000000,1.000000,23.000000,24.000000


# Mice imputation

The mean of the **RCRI score** almost remained the same from 0.322 to 0.321

The mean of the **PreopEGFRMDRD** almost remained the same from 96.426 to 96.703

In [ ]:
# Read original dataset first
df = pd.read_csv("CARES_data_cleaned.csv")
df

#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())


27424
10830


In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.impute import SimpleImputer


# Step 1: Impute missing values in PreopEGFRMDRD using mean
imputer = SimpleImputer(strategy='mean')
df['PreopEGFRMDRD'] = imputer.fit_transform(df[['PreopEGFRMDRD']])

# Step 2: Regression imputation for RCRI score
regressor_rcri = LinearRegression()
non_na_data_rcri = df.dropna(subset=['RCRI score', 'PreopEGFRMDRD'])
regressor_rcri.fit(non_na_data_rcri[['PreopEGFRMDRD']], non_na_data_rcri['RCRI score'])

missing_rcri = df[df['RCRI score'].isna()]
if not missing_rcri.empty:
    df.loc[df['RCRI score'].isna(), 'RCRI score'] = regressor_rcri.predict(missing_rcri[['PreopEGFRMDRD']])

# Check if there are still missing values in PreopEGFRMDRD after mean imputation
if df['PreopEGFRMDRD'].isna().sum() > 0:
    # Regression imputation for PreopEGFRMDRD
    regressor_preop = LinearRegression()
    non_na_data_preop = df.dropna(subset=['PreopEGFRMDRD', 'RCRI score'])
    regressor_preop.fit(non_na_data_preop[['RCRI score']], non_na_data_preop['PreopEGFRMDRD'])

    missing_preop = df[df['PreopEGFRMDRD'].isna()]
    if not missing_preop.empty:
        df.loc[df['PreopEGFRMDRD'].isna(), 'PreopEGFRMDRD'] = regressor_preop.predict(missing_preop[['RCRI score']])


In [ ]:
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())

0
0


In [ ]:
df.describe()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop
count,90785.000000,90785.000000,90785.000000,90785.000000,5595.000000,90785.000000,90785.000000,90785.000000,90785.000000
mean,60334.001057,52.253225,0.320926,96.426155,476.782127,0.042672,0.056276,0.018946,0.075222
std,34930.467992,17.087307,0.539922,31.864673,421.866740,0.405840,0.230455,0.260728,0.395623
min,2.000000,18.000000,-1.856552,2.541026,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29973.000000,39.000000,0.000000,81.773853,121.000000,0.000000,0.000000,0.000000,0.000000
50%,60572.000000,54.000000,0.000000,96.426155,355.000000,0.000000,0.000000,0.000000,0.000000
75%,90788.000000,65.000000,0.402673,111.439680,746.000000,0.000000,0.000000,0.000000,0.000000
max,121200.000000,103.000000,6.000000,671.298147,1783.000000,21.000000,1.000000,23.000000,24.000000


# Regression Imputation

The mean of the **RCRI score** slightly decreased from 0.322 to 0.321

The mean of the **PreopEGFRMDRD** remained the same from 96.426 to 96.426

In [ ]:
# Read original dataset first
df = pd.read_csv("CARES_data_cleaned.csv")
df

#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())


27424
10830


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Step 1: Impute missing values in PreopEGFRMDRD using mean
imputer = SimpleImputer(strategy='mean')
df['PreopEGFRMDRD'] = imputer.fit_transform(df[['PreopEGFRMDRD']])

# Step 2: Random forest imputation for RCRI score
rf_imputer_rcri = RandomForestRegressor(n_estimators=100)
non_na_data_rcri = df.dropna(subset=['RCRI score', 'PreopEGFRMDRD'])
rf_imputer_rcri.fit(non_na_data_rcri[['PreopEGFRMDRD']], non_na_data_rcri['RCRI score'])

missing_rcri = df[df['RCRI score'].isna()]
if not missing_rcri.empty:
    df.loc[df['RCRI score'].isna(), 'RCRI score'] = rf_imputer_rcri.predict(missing_rcri[['PreopEGFRMDRD']])

# Check if there are still missing values in PreopEGFRMDRD after mean imputation
if df['PreopEGFRMDRD'].isna().sum() > 0:
    # Random forest imputation for PreopEGFRMDRD
    rf_imputer_preop = RandomForestRegressor(n_estimators=100)
    non_na_data_preop = df.dropna(subset=['PreopEGFRMDRD', 'RCRI score'])
    rf_imputer_preop.fit(non_na_data_preop[['RCRI score']], non_na_data_preop['PreopEGFRMDRD'])

    missing_preop = df[df['PreopEGFRMDRD'].isna()]
    if not missing_preop.empty:
        df.loc[df['PreopEGFRMDRD'].isna(), 'PreopEGFRMDRD'] = rf_imputer_preop.predict(missing_preop[['RCRI score']])

In [ ]:
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())

0
0


In [ ]:
df.describe()

,Indexno,Age,RCRI score,PreopEGFRMDRD,DaysbetweenDeathandoperation,Preoptransfusionwithin30days,Intraop,Postopwithin30days,Transfusionintraandpostop
count,90785.000000,90785.000000,90785.000000,90785.000000,5595.000000,90785.000000,90785.000000,90785.000000,90785.000000
mean,60334.001057,52.253225,0.317959,96.426155,476.782127,0.042672,0.056276,0.018946,0.075222
std,34930.467992,17.087307,0.575790,31.864673,421.866740,0.405840,0.230455,0.260728,0.395623
min,2.000000,18.000000,0.000000,2.541026,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29973.000000,39.000000,0.000000,81.773853,121.000000,0.000000,0.000000,0.000000,0.000000
50%,60572.000000,54.000000,0.000000,96.426155,355.000000,0.000000,0.000000,0.000000,0.000000
75%,90788.000000,65.000000,0.345924,111.439680,746.000000,0.000000,0.000000,0.000000,0.000000
max,121200.000000,103.000000,6.000000,671.298147,1783.000000,21.000000,1.000000,23.000000,24.000000


# Random Forest Imputation

The mean of the **RCRI score** slightly decreased from 0.322 to 0.318

The mean of the **PreopEGFRMDRD** remained the same from 96.426 to 96.426

In [ ]:
"""
# Read original dataset first
df = pd.read_csv("CARES_data_cleaned.csv")
df

#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())
"""

'\n# Read original dataset first\ndf = pd.read_csv("CARES_data_cleaned.csv")\ndf\n\n#Check that there are no more missing values in these two columns:\nprint(df[\'RCRI score\'].isna().sum())\nprint(df[\'PreopEGFRMDRD\'].isna().sum())\n'

In [ ]:
"""
from sklearn.impute import KNNImputer
import pandas as pd

# Using KNN for imputation
knn_imputer = KNNImputer(n_neighbors=1)
df_imputed = df.copy()
imputed_data = knn_imputer.fit_transform(df[['RCRI score', 'PreopEGFRMDRD']])

# Replace NaN values with computed values
df['RCRI score'] = imputed_data[:, 0]
df['PreopEGFRMDRD'] = imputed_data[:, 1]

# Display the first few rows to verify imputation
print(df.head())
"""

"\nfrom sklearn.impute import KNNImputer\nimport pandas as pd\n\n# Using KNN for imputation\nknn_imputer = KNNImputer(n_neighbors=1)\ndf_imputed = df.copy()\nimputed_data = knn_imputer.fit_transform(df[['RCRI score', 'PreopEGFRMDRD']])\n\n# Replace NaN values with computed values\ndf['RCRI score'] = imputed_data[:, 0]\ndf['PreopEGFRMDRD'] = imputed_data[:, 1]\n\n# Display the first few rows to verify imputation\nprint(df.head())\n"

In [ ]:
"""
#Check that there are no more missing values in these two columns:
print(df['RCRI score'].isna().sum())
print(df['PreopEGFRMDRD'].isna().sum())
"""

"\n#Check that there are no more missing values in these two columns:\nprint(df['RCRI score'].isna().sum())\nprint(df['PreopEGFRMDRD'].isna().sum())\n"

In [ ]:
"""
df.describe()
"""

'\ndf.describe()\n'

# K-Nearest Neighbors (KNN) Imputation - pending